# 🦌 Node Menagerie ⚡

Demonstrating the variety of Node Shapes and introduce the `Symbol` classes:

- Rect - A rectangular node
- Circle - A circlular node
- Ellipse - An elliptical node
- Diamond - A diamond node
- Comment - A mostly rectangular node that has been notched in the upper right corner
- Image - Wrapper for a `image` svg tag
- Path - Wrapper for a `path` svg tag
- SVG - embed raw svg

- Widget - embed other jupyterlab widgets
- HTML - embed a html sting

These classes help to generate the valid ElkJSON structures that can be added to a
networkx graphs and passed to the Elk display widgets. The generated JSON has
`properties` -> `type` set to a value that will be inspected on the frontend to render
the appropriate shape. When calling a shapes `to_json` method an `id` arguement should
be supplied and will be included in the output dictionary.

In [ ]:
import importnb

# import ipyelk.diagram.elk_export
import ipywidgets as W
import networkx as nx
from IPython.display import SVG

import ipyelk
import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools

# from ipyelk.contrib.library import logic_gates as logic
from ipyelk import Elk
from ipyelk.contrib.shapes import connectors as conn
from ipyelk.contrib.shapes import shapes
from ipyelk.diagram import elk_export, elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram import symbol
from ipyelk.diagram.defs import ConnectorDef, Def
from ipyelk.diagram.layout_options.layout import ELKRectanglePacking

### Symbol Example

In [ ]:
def symbol_example():
    g = nx.Graph()
    tree = nx.DiGraph()

    width = 150
    height = 100

    symbols = [
        symbol.Image(
            value="./files/examples/untitled_example.svg",
        ),
        symbol.SVG(
            value="""<g><rect fill="none" height="24" width="24"/><path d="M17,15l1.55,1.55c-0.96,1.69-3.33,3.04-5.55,3.37V11h3V9h-3V7.82C14.16,7.4,15,6.3,15,5c0-1.65-1.35-3-3-3S9,3.35,9,5 c0,1.3,0.84,2.4,2,2.82V9H8v2h3v8.92c-2.22-0.33-4.59-1.68-5.55-3.37L7,15l-4-3v3c0,3.88,4.92,7,9,7s9-3.12,9-7v-3L17,15z M12,4 c0.55,0,1,0.45,1,1s-0.45,1-1,1s-1-0.45-1-1S11.45,4,12,4z"/></g>"""
        ),
        symbol.Path(
            value="M 0,0 L 0,100 L 20,30 Z",
        ),
        symbol.ForeignObject(value="<input/>"),
        symbol.Rect(),
        symbol.Diamond(),
        symbol.Comment(),
        symbol.Circle(radius=width / 2),
        symbol.Ellipse(rx=width / 2, ry=height / 2),
        symbol.HTML(value="<h1>Hello World</h1>"),
    ]

    for i, s in enumerate(symbols):
        id = str(i)
        s.width = width
        s.height = height
        data = s.to_json(id=id)
        data["labels"] = [s.__class__.__name__]
        g.add_node(id, **data)

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(
            layouts={
                ipyelk.diagram.elk_model.ElkRoot: {
                    "parents": opt.OptionsWidget(
                        options=[
                            opt.LayoutAlgorithm(),
                            opt.HierarchyHandling(),
                            opt.NodeLabelPlacement(),
                        ]
                    ).value,
                }
            },
            source=(g, tree),
        ),
        layout={"height": "100%"},
    )
    return app

In [ ]:
if __name__ == "__main__":
    symbol_app = symbol_example()
    display(symbol_app)

### Widget Example

Demonstate how to attach jupyterlab widgets for use in diagram nodes.

In [ ]:
def widget_example():
    slider_box = W.VBox(children=[W.FloatSlider() for i in range(5)])

    g = nx.Graph()
    tree = nx.DiGraph()

    g.add_node("sliders")

    g.add_node(
        "slider_box",
        **symbol.Widget(width=320, height=200, widget=slider_box).to_json(
            id="slider_box"
        )
    )
    g.add_edge("sliders", "slider_box")

    for i, slider in enumerate(slider_box.children):
        id = str(i)
        g.add_node(
            id, **symbol.Widget(width=320, height=70, widget=slider).to_json(id=id)
        )
        tree.add_edge("sliders", id)

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(
            layouts={
                ipyelk.diagram.elk_model.ElkRoot: {
                    "parents": opt.OptionsWidget(
                        options=[
                            opt.LayoutAlgorithm(),
                            opt.HierarchyHandling(),
                            opt.NodeLabelPlacement(),
                        ]
                    ).value,
                }
            },
            source=(g, tree),
        ),
        layout={"height": "100%"},
    )
    toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    return app

In [ ]:
if __name__ == "__main__":
    widget_app = widget_example()
    display(widget_app)

### Nesting Diagrams

In [ ]:
def nesting_diagram_example():
    # build graphs
    slider_diagram = widget_example()
    g = nx.Graph()
    tree = nx.DiGraph()

    g.add_node("n1", width=40, height=40)
    tree.add_node("slider_diagram", hidden=True)

    g.add_node(
        "slider_diagram",
        **symbol.Widget(width=400, height=700, widget=slider_diagram).to_json(
            id="slider_diagram"
        )
    )
    tree.add_edge("n1", "slider_diagram")

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(
            layouts={
                ipyelk.diagram.elk_model.ElkRoot: {
                    "parents": opt.OptionsWidget(
                        options=[
                            opt.LayoutAlgorithm(),
                            opt.HierarchyHandling(),
                            opt.NodeLabelPlacement(),
                        ]
                    ).value,
                }
            },
            source=(g, tree),
        ),
        layout={"height": "100%"},
    )
    toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    return app

In [ ]:
if __name__ == "__main__":
    nested_app = nesting_diagram_example()
    display(nested_app)